# Exploring CIA Factbook Data Using SQL

The [CIA World Factbook](https://www.cia.gov/the-world-factbook/) is a comprehensive guide that provides information on the geography, people, government, economy, energy, communications, transportation, military, and transnational issues of countries and territories around the world.

We will be exploring this dataset through SQLite queries in this notebook, which was done as a project on Dataquest.io. The site (DQ) did not provide information on when this dataset was downloaded. See further comments at the end of this notebook. 


In [2]:
# Install ipython-sql to run sql on python and jupyter notebook 
!conda install -yc conda-forge ipython-sql

Retrieving notices: ...working... done
Solving environment: done

# All requested packages already installed.



Use the code to connect Jupyter Notebook to our database file: 

In [3]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [4]:
%%sql --should be in every cell to specify its an sql query.
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Data Overview
Select the first five rows of the `facts` table. 

In [5]:
%%sql 
SELECT * 
    FROM facts
  LIMIT 5; 

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### Data Dictionary

| column | details | 
| :-: | :-: |
| name | the name of the country. |
| area | the country's total area (both land and water). |
| area_land | the country's land area in square kilometers. |
| area_water | the country's waterarea in square kilometers. |
| population | the country's population. |
| population_growth | the country's population growth as a percentage. |
| birth_rate |  the country's birth rate, or the number of births per year  per 1,000 people. |
| death_rate | the country's death rate, or the number of death per year per 1,000 people. |


### Summary Statistics
Let's look at an overview of the population in the factbook database. 

In [6]:
%%sql 
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts; 

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


There is a country with no population, and no population growth. There is a country with a population of 7,256,490,011 and a maximum population growth of 4.02. However, the 7 billion population is likely the earth's total population 

Let's create a query that returns the countries with the minimum population, and maximum population. 

In [7]:
%%sql 
SELECT *, min(population)
    FROM facts;


 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,min(population)
250,ay,Antarctica,None,280000,None,0,None,None,None,None,0


In [8]:
%%sql 
SELECT *, max(population)
    FROM facts; 

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,max(population)
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None,7256490011


### Exploring Average Population and Area

The Summary Statistics should be recalculated with the whole world population excluded.


In [9]:
%%sql 
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts
  WHERE population < 7256490011; 


 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


#### What are the countries that have above average populations but areas less than the average? 

In [16]:
%%sql 
SELECT *
    FROM facts
  WHERE population > (SELECT AVG(population) FROM facts)
  AND area < (SELECT AVG(area) FROM facts)
  ORDER BY population DESC; 


 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54


#### Which country has the most people? 


In [11]:
%%sql 
SELECT name, MAX(population) 
    FROM facts
  WHERE name <> 'World' ; 

 * sqlite:///factbook.db
Done.


name,MAX(population)
China,1367485388


#### Which countries have the highest ratios of water to land? 

In [37]:
%%sql 
--more water than land
SELECT name, ROUND(CAST (area_water AS FLOAT) / area, 4) AS Water_land_ratio
    FROM facts
  WHERE Water_land_ratio > 0
  ORDER BY Water_land_ratio DESC
  LIMIT 5; 



 * sqlite:///factbook.db
Done.


name,Water_land_ratio
British Indian Ocean Territory,0.9989
Virgin Islands,0.8188
Puerto Rico,0.3568
"Bahamas, The",0.2788
Guinea-Bissau,0.2216


#### Which countries will add the most people to their populations next year?



In [18]:
%%sql 

SELECT name, population, ROUND(population_growth/100,5) AS population_growth, CAST((population * population_growth/100) AS Integer) AS population_increase
    FROM facts
  WHERE name <> 'World'
  ORDER BY population_increase DESC
  LIMIT 5;
  

 * sqlite:///factbook.db
Done.


name,population,population_growth,population_increase
India,1251695584,0.0122,15270686
China,1367485388,0.0045,6153684
Nigeria,181562056,0.0245,4448270
Pakistan,199085847,0.0146,2906653
Ethiopia,99465819,0.0289,2874562


#### Which countries have a higher death rate than birth rate?

In [19]:
%%sql 

SELECT name, death_rate, birth_rate, round((death_rate-birth_rate),2) AS rate_difference
    FROM facts
  WHERE death_rate > birth_rate
  ORDER BY rate_difference DESC
  LIMIT 10; 

 * sqlite:///factbook.db
Done.


name,death_rate,birth_rate,rate_difference
Bulgaria,14.44,8.92,5.52
Serbia,13.66,9.08,4.58
Latvia,14.31,10.0,4.31
Lithuania,14.27,10.1,4.17
Ukraine,14.46,10.72,3.74
Hungary,12.73,9.16,3.57
Germany,11.42,8.47,2.95
Slovenia,11.37,8.42,2.95
Romania,11.9,9.14,2.76
Croatia,12.18,9.45,2.73


#### Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

In [20]:
%%sql 
SELECT name, population/area as population_area_ratio
    FROM facts
  WHERE name <> 'World'
  ORDER BY population_area_ratio DESC
  LIMIT 10 ; 

 * sqlite:///factbook.db
Done.


name,population_area_ratio
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299


### Conclusion
* Bangladesh has the largest population among territories with less than average area. 
* China has the largest population in the world
* The British Indian Ocean Territory and Virgin Islands have more water-to-land ratio.
* India and China are the top two countries that will increase its population next year.
* Macau and Monaco have the highest population-to-area ratios. 
* Most countries with higher death rates than birth rates are European countries and Russia, while countries with the highest population to area ratios are Asian and African countries.

### Further Comments: 
* This is completed as part of a guided project through Dataquest.io. It is worth noting that the dataset may be altered or truncated for this guided project, since a quick google search of the country with largest water-to-land ratio appears to be the Maldives, but its water area is recorded to be 0 in this dataset.  


In [38]:
%%sql
SELECT *
    FROM facts
  WHERE name = 'Maldives';

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
108,mv,Maldives,298,298,0,393253,0.08,15.75,3.89,12.68
